In [1]:
# loading R packages
library(DESeq2)
library(phyloseq)
library(plyr)
library(dplyr)
library(ggplot2)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The following objects are maske

In [2]:
# Importing the dataset
ps = import_biom("../OTU_table/feature-table-metaD-tax.biom",parseFunction=parse_taxonomy_greengenes, "../OTU_table/Trees/fulltree.tre")

Warning message in strsplit(msg, "\n"):
“input string 1 is invalid in this locale”

In [3]:
# Setting various parameters as combined values
sample_data(ps)$pH = as.numeric(sample_data(ps)$pH)
sample_data(ps)$Total_C_pct = as.numeric(sample_data(ps)$TC_pct)
sample_data(ps)$Total_N_pct = as.numeric(sample_data(ps)$Total_N_pct)
sample_data(ps)$Total_S_pct = as.numeric(sample_data(ps)$Total_S_pct)
sample_data(ps)$Burn_Severity_Index= as.numeric(sample_data(ps)$Burn_Severity_Index)
sample_data(ps)$CBI= as.numeric(sample_data(ps)$CBI)
sample_data(ps)$RBR= as.numeric(sample_data(ps)$RBR)
sample_data(ps)$CFSI= as.numeric(sample_data(ps)$CFSI)
sample_data(ps)$Mean_Duff_Depth_cm = as.numeric(sample_data(ps)$Mean_Duff_Depth_cm)
sample_data(ps)$Understory_CBI= as.numeric(sample_data(ps)$Understory_CBI)
sample_data(ps)$Overstory_CBI= as.numeric(sample_data(ps)$Overstory_CBI)
sample_data(ps)$Overstory_CBI= as.numeric(sample_data(ps)$Overstory_CBI)
sample_data(ps)$Dead_Trees= as.numeric(sample_data(ps)$Dead_Trees)
sample_data(ps)$Pct_Exposed_Mineral= as.numeric(sample_data(ps)$Pct_Exposed_Mineral)
sample_data(ps)$dc = as.numeric(sample_data(ps)$dc)
sample_data(ps)$fwi = as.numeric(sample_data(ps)$fwi)
sample_data(ps)$dmc = as.numeric(sample_data(ps)$dmc)
sample_data(ps)$ffmc = as.numeric(sample_data(ps)$ffmc)
sample_data(ps)$TotalSeqs=sample_sums(ps)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, e

In [4]:
# Pulling out samples from this project only
ps.wb = prune_samples(sample_data(ps)$Project_ID=="WB15", ps)
ps.wb = prune_samples(sample_data(ps.wb)$Org_or_Min %in% c("O","M"), ps.wb)
# Merging lab replicates for analysis
ps.merged = merge_samples(ps.wb, "Sample_ID")

# This sums the OTUs for each sample ID, and averages the sample data
# That's an issue for the non-numeric sample data, so we have to add it back in.
# The code below is probably inefficient, but it works correctly.
names=colnames(sample_data(ps.wb))
notcommonnames = c('Sample_Name','Replicate','Project_ID','Fwd_Primer_Barcode','Rev_Primer_Barcode','Revcomp_Rev_Primer_Barcode')
commonnames = names[(names %in% notcommonnames)]
common_sample_data = sample_data(ps.wb)[ , -which(names(sample_data(ps.wb)) %in% commonnames)]
commonrownames=row.names(sample_data(ps.merged))
common_sample_data2 = common_sample_data[which(common_sample_data$Sample_ID %in% commonrownames),]
common_sample_data2 = common_sample_data2[!duplicated(common_sample_data2$Sample_ID), ]
row.names(common_sample_data2) = common_sample_data2$Sample_ID
sample_data(ps.merged)=common_sample_data2

minseqs = 1500
ps.merged.min = prune_samples(sample_data(ps.merged)$TotalSeqs>minseqs,ps.merged)

Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMethod(object):
“NAs introduced by coercion”Warning message in asMeth

In [7]:
ps = ps.merged.min
ps = prune_samples(!is.na(sample_data(ps)$CBI), ps)
ps = prune_samples(!is.na(sample_data(ps)$pH), ps)
ps = prune_samples(!is.na(sample_data(ps)$Org_or_Min), ps)
ps = prune_taxa(taxa_sums(ps)>0, ps)
ps

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 7612 taxa and 93 samples ]
sample_data() Sample Data:       [ 93 samples by 66 sample variables ]
tax_table()   Taxonomy Table:    [ 7612 taxa by 7 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 7612 tips and 7609 internal nodes ]

In [6]:
ps = ps.merged.min
ps = prune_samples(!is.na(sample_data(ps)$CBI), ps)
#ps = prune_samples(!is.na(sample_data(ps)$pH), ps)
#ps = prune_samples(!is.na(sample_data(ps)$Org_or_Min), ps)
#ps = prune_taxa(taxa_sums(ps)>2000, ps)
# Trying different cutoffs to help estimateSizeFactors work

dseq = phyloseq_to_deseq2(ps, ~CBI)
dseq = estimateSizeFactors(dseq, type="iterate")

converting counts to integer mode


ERROR: Error in estimateSizeFactorsIterate(object): iterative size factor normalization did not converge


In [28]:
# This is done because all "genes" contain at least one zero - need to get t
#dseq$Trtmt = relevel(dseq$Trtmt,"Soil")
dseq = DESeq(dseq, quiet = FALSE, fitType = "local")
results = results(dseq, cooksCutoff=TRUE)
# Change Cooks Cutoff to control outliers or not
results$CBI = CBI
results$pH = pH
results$Org_or_Min = Org_or_Min
taxonomy = data.frame(tax_table(ps))
results = data.frame(results$CBI,results$pH,results$Org_or_Min,rownames(results),results$baseMean,results$log2FoldChange,results$lfcSE,results$pvalue,taxonomy[,1:7])
colnames(results) = c("CBI","pH","Org_or_Min","OTU","baseMean","l2FC","SE","pvalue","Kingdom","Phylum","Class","Order","Family","Genus","Species")

converting counts to integer mode
Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”

ERROR: Error in estimateSizeFactorsIterate(object): iterative size factor normalization did not converge


In [ ]:
DA = results

In [ ]:
threshold = function (thresh){
    dplyr::filter(DA, baseMean >= thresh) %>% 
    dplyr::mutate(padj = p.adjust(pvalue,"BH")) %>%
    dplyr::summarize(cutoff=thresh, count=sum(padj<=0.10, na.rm = TRUE))
}

In [ ]:
range = seq(0,1,0.05)
# Creates a range of numbers we are interested in for adjusted p values
thresh.s <- ldply(range, threshold)
# Applys the Threshold function we created above to the range of numbers we created above.

plot(thresh.s$count~thresh.s$cutoff)
# We can plot the threshold for base Mean value against the number of samples that will pass under this cutoff.

In [ ]:
group_by(DA, Trtmt,Month) %>%
    mutate(Total=sum(baseMean)) %>%
    filter(baseMean>=0.6) %>% 
    mutate(padj = p.adjust(pvalue,"BH")) %>% 
    write.csv("", row.names=FALSE)
# We take that differential abundance table we created above, and filter it to include only those samples 
# that had a baseMean value above the optimal cutoff we determined above.
# We then adjust the p values and save it as a csv.

In [ ]:
l2fc = read.csv("../../SeqData/l2fc.csv")

In [ ]:
cutoff = 0.1

d = l2fc
d = group_by(d, Trtmt, Month) %>%
    mutate(sig = ifelse(padj<cutoff,1,0))%>%
    mutate(relabund=baseMean/Total)%>%
    filter(pvalue != 'NA')

p = ggplot(d, aes(x = Phylum, y = l2FC, fill = Phylum, size = relabund))
# establishing our plot is based on the data table d, with our x being the phylum, and the y being log2fold change,
# and the colour being based on our "yes" or "no" column in the d data table.

p = p + facet_grid(~Land_Class~Org_or_Min, scales="free_x")
# saying we want it to present the data separately for our two variables (trtmt and month), in 2 columns

d$sig = as.factor(d$sig)
    
d.yes = d[d$sig==1,]
d.no = d[d$sig==0,]
    
p = p + geom_point(data=d.yes, aes(x = Phylum, y = l2FC, fill = Phylum, size=relabund), shape = 21, alpha=0.4, position = position_jitter(w = 0.20))
p = p + geom_point(data=d.no, aes(x = Phylum, y = l2FC, fill = sig, size=relabund), shape = 21, alpha=0.3, position = position_jitter(w = 0.20))

    
#p = p + geom_point(shape = 21, alpha=0.45, position = position_jitter(w = 0.20), aes(size = relabund))
# sets the points size and shape, and offsets them slightly (jitter) so they're not right on top of each other

p = p + scale_size_continuous("log(Relative\nAbundance)",trans="log",guide="none")

p = p + theme_bw()
# sets a theme

p = p + theme(strip.text.x = element_text(size = 24),
              strip.text.y = element_text(size = 24),
              axis.text.x = element_text(size = 14, angle = 45, hjust = 1, vjust = 1),
              axis.title.x = element_text(size = 28),
              axis.text.y = element_text(size=16),
              axis.title.y = element_text(size = 28),
              legend.title = element_text(size=20),
              legend.text = element_text(size = 14),
              #legend.position = "none",
              strip.background = element_blank())
# sets a bunch of visual paramters for the legend (none) and other text
    
p = p + labs(x = "Phylum")
# sets the label for the x axis. I think they didn't get the different colours.

p = p + labs(y = expression(paste("", log[2]," fold change with increasing CBI",sep="")))
# sets the label for the y axes.
p = p + scale_fill_manual(guide="none", values=c("grey",paste(getPalette(colourCount))))


#p = p + scale_color_continuous(trans="log")    
#p = p + scale_fill_discrete(guide="none")


p = p + theme(panel.grid.major = element_blank(), panel.grid.minor = element_blank())
# Gets rid of the default gridlines

    
# Would be cool to colour this by relative abundance in soil to begin with.. what drives patterns more?
    
p